# Exercise 2: Delivery Risk Assessment with External AI Models

In the previous exercise, we collected comprehensive delivery data by orchestrating multiple agents. Now we'll take that data and perform sophisticated risk assessment.

## The Big Picture: Building Toward Intelligent Case Cards

We're building a complete **Delivery Intelligence System** that helps prevent delivery failures. Here's how this exercise fits in:

```
1. Data Collection (Exercise 1) → collected_order_data.json
   ↓
2. Risk Assessment (THIS EXERCISE) → risk_assessment_output.json
   ↓
3. Product Intelligence (Exercise 3) → product_analysis.json
   ↓
4. Communication & Cases (Exercise 4) → final_case_card.json
```

### What We'll Build in This Exercise

1. **External AI Model Integration** - Calling a risk assessment model (simulating your proprietary model)
2. **Multi-Factor Risk Analysis** - Weather, customer, and route risk factors
3. **MCP Integration** - Using real weather data via Model Context Protocol
4. **Risk Aggregation** - Combining all assessments into structured insights

### The Final Goal

By the end of all exercises, we'll produce intelligent case cards that GOAs can use to:
- Identify high-risk deliveries before they fail
- Send pre-drafted, policy-compliant messages to customers
- Communicate specific requirements to carriers
- Suggest alternative delivery solutions

This exercise provides the **risk intelligence** that drives prioritization and recommendations.

## Environment Setup

We'll set up the environment and import necessary libraries, including Vertex AI for model integration:

In [ ]:
import os
import warnings
import logging
import json

# Suppress warnings for clean output
warnings.filterwarnings("ignore")
logging.getLogger().setLevel(logging.ERROR)

# Configure environment
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"
os.environ["GOOGLE_CLOUD_PROJECT"] = "traversaal-research"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"

from google.adk.agents import Agent, SequentialAgent, ParallelAgent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types

print("✅ Environment configured for risk assessment")

## Understanding the Data Flow

In this exercise, we'll use the actual data collected from Exercise 1. The data collection pipeline saves its output to `collected_order_data.json`, which we'll load and use for risk assessment.

This creates a realistic data pipeline where:
- Exercise 1 collects data from BigQuery → `collected_order_data.json`
- Exercise 2 loads that data and performs risk assessment → `risk_assessment_output.json`
- Exercise 3 will use both outputs for product intelligence
- Exercise 4 will combine everything for communication generation

Let's load the data from Exercise 1:

In [ ]:
# Load the order data from Exercise 1
def load_order_data(file_path: str = '../exercise_1_data_collection/collected_order_data.json'):
    """Load order data from Exercise 1 output"""
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"⚠️ Warning: {file_path} not found.")
        print("Please run Exercise 1 first to generate the data.")
        print("Using fallback sample data for demonstration...")
        
        # Fallback data if Exercise 1 hasn't been run
        return {
            "order": {
                "DATA_ID": 2204,
                "CUSTOMER_ORDER_NUMBER": "CG92094171",
                "SCHEDULED_DELIVERY_DATE": "2025-06-21T00:00:00",
                "VEHICLE_TYPE": "FLAT",
                "QUANTITY": 109,
                "VOLUME_CUBEFT": 34.9,
                "WEIGHT": 1598,
                "PALLET": 3,
                "WINDOW_START": "06:00:00",
                "WINDOW_END": "20:00:00"
            },
            "customer": {
                "CUSTOMER_NAME": "CUST_01518",
                "PRO_XTRA_MEMBER": True,
                "COMMERCIAL_ADDRESS_FLAG": False,
                "DESTINATION_ADDRESS": "668 FOREST AVE ELGIN, IL 60120",
                "CUSTOMER_NOTES": "call b/4 delivery delivery from the back of the building"
            },
            "products": [
                "2 in. x 12 in. x 8 ft. 2 Prime Ground Contact Pressure-Treated Lumber",
                "2 in. x 4 in. x 12 ft. 2 Prime Ground Contact Pressure-Treated Southern Yellow Pine Lumber"
            ],
            "environmental": {
                "WTHR_CATEGORY": "Clear",
                "PRECIPITATION": "0.09 inch",
                "STRT_VW_IMG_DSCRPTN": "* The driveway is partially obscured by trees"
            },
            "risk_info": {
                "DLVRY_RISK_DECILE": 6,
                "DLVRY_RISK_BUCKET": "MEDIUM",
                "DLVRY_RISK_PERCENTILE": 40,
                "DLVRY_RISK_TOP_FEATURE": "WORK_ORD_TOTAL,LUMBER_CNT,IS_SPECIFIC_DLVRY_WINDOW"
            }
        }

# Load the actual order data
order_data = load_order_data()

print("📦 Order Summary:")
print(f"  - Order #: {order_data['order']['CUSTOMER_ORDER_NUMBER']}")
print(f"  - Weight: {order_data['order']['WEIGHT']} lbs")
print(f"  - Products: {len(order_data['products'])} items")
print(f"  - Customer: PRO Member = {order_data['customer']['PRO_XTRA_MEMBER']}")
print(f"  - Risk Level (Pre-calculated): {order_data.get('risk_info', {}).get('DLVRY_RISK_BUCKET', 'Unknown')}")

# Display customer notes if present
customer_notes = order_data['customer'].get('CUSTOMER_NOTES')
if customer_notes:
    print(f"  - Special Instructions: {customer_notes}")
else:
    print("  - Special Instructions: None")

## Model Context Protocol (MCP) Integration

Before we create our risk assessment tools, let's explore a powerful ADK feature: **Model Context Protocol (MCP)**.

### What is MCP?

MCP is an open standard that allows LLMs to communicate with external applications and services through a standardized protocol. Think of it as a universal adapter that lets your agents connect to any service that speaks MCP.

### Why use MCP in our Risk Assessment?

In our current implementation, we use mock weather data. But in production, you'd want **real weather forecasts** for the delivery date. MCP allows us to:

1. **Connect to external services** without writing custom integration code
2. **Use community-built MCP servers** or create our own
3. **Maintain clean separation** between our agent logic and external services
4. **Easily swap services** without changing agent code

Let's first see an example of a weather MCP server, then use it in our risk assessment!

In [ ]:
# First, let's see the structure of a simple MCP server for weather
# In production, you'd have this as a separate service

weather_mcp_server_code = '''#!/usr/bin/env python3
"""
Simple Weather MCP Server - Demonstrates MCP integration for ADK

This server provides:
1. get_weather - Get weather for a city/date
2. assess_weather_risk - Assess delivery risk based on weather
"""

from mcp.server import Server, NotificationOptions
from mcp.server.models import InitializationOptions
from mcp.server.stdio import stdio_server
from mcp import types
import json
import asyncio

app = Server("weather-server")

# For demo: simulated weather data
DEMO_WEATHER_DATA = {
    "chicago": {
        "temperature": 72,
        "conditions": "Partly Cloudy",
        "precipitation": 0.0,
        "wind_speed": 8,
        "humidity": 65
    },
    "new york": {
        "temperature": 68,
        "conditions": "Clear",
        "precipitation": 0.0,
        "wind_speed": 5,
        "humidity": 55
    }
}

@app.list_tools()
async def list_tools() -> list[types.Tool]:
    """List available weather tools"""
    return [
        types.Tool(
            name="assess_weather_risk",
            description="Assess delivery risk based on weather conditions",
            inputSchema={
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "City name"},
                    "date": {"type": "string", "description": "Delivery date"}
                },
                "required": ["city", "date"]
            }
        )
    ]

@app.call_tool()
async def call_tool(name: str, arguments: Any) -> list[types.TextContent]:
    """Handle tool calls"""
    if name == "assess_weather_risk":
        city = arguments.get("city", "").lower()
        weather = DEMO_WEATHER_DATA.get(city, DEMO_WEATHER_DATA["chicago"])
        
        # Risk assessment logic
        risk_score = 1
        risk_factors = []
        
        if weather["precipitation"] > 0.5:
            risk_score = 6
            risk_factors.append("Precipitation")
        elif "storm" in weather["conditions"].lower():
            risk_score = 8
            risk_factors.append("Storm conditions")
        else:
            risk_factors.append("Favorable weather")
            
        return [types.TextContent(
            type="text",
            text=json.dumps({
                "weather_risk_score": risk_score,
                "weather_factors": risk_factors,
                "weather_data": weather,
                "risk_level": "HIGH" if risk_score >= 7 else "MEDIUM" if risk_score >= 4 else "LOW"
            }, indent=2)
        )]
'''

# Save the server code for reference
with open('weather_mcp_server_demo.py', 'w') as f:
    f.write(weather_mcp_server_code)

print("✅ Weather MCP Server example created")
print("\nKey MCP concepts demonstrated:")
print("1. Server definition with app = Server()")
print("2. Tool listing with @app.list_tools()")
print("3. Tool execution with @app.call_tool()")
print("4. Structured input/output schemas")

### Understanding the MCP Server Structure

The weather MCP server above demonstrates key concepts:

1. **Server Definition**: `app = Server("weather-server")` creates the MCP server
2. **Tool Registration**: `@app.list_tools()` defines what tools are available
3. **Tool Implementation**: `@app.call_tool()` handles tool execution
4. **Structured I/O**: Clear input schemas and JSON output

This server can be run as a separate process and our ADK agents can connect to it using the `MCPToolset`.

## Integrating External AI Models

One of ADK's strengths is integrating with existing AI models. Your organization likely has proprietary risk models that you want to keep using.

Here we'll create a tool that:
1. Calls an external AI model (we'll use pre-calculated BigQuery data to simulate your model)
2. Returns risk assessments in **your exact format**
3. Can easily be swapped with your actual model endpoint

The model returns:
- `DLVRY_RISK_DECILE`: 1-10 scale (10 = highest risk)
- `DLVRY_RISK_BUCKET`: HIGH/MEDIUM/LOW categorization
- `DLVRY_RISK_PERCENTILE`: 0-100 percentile ranking
- `DLVRY_RISK_TOP_FEATURE`: Key risk factors identified

In [ ]:
from typing import Dict, Any
import vertexai
from vertexai.generative_models import GenerativeModel
import google.auth

# Initialize Vertex AI
credentials, project = google.auth.default()
vertexai.init(project=project, location="us-central1")

def call_external_risk_model(order_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Call external risk assessment model.
    In production, this would call the client's proprietary model.
    For the workshop, we'll use the pre-calculated risk data from BigQuery.
    """
    # Extract pre-calculated risk info from the order data
    risk_info = order_data.get('risk_info', {})
    
    # Map the BigQuery risk data to the expected format
    risk_bucket = risk_info.get('DLVRY_RISK_BUCKET', 'MEDIUM')
    risk_decile = risk_info.get('DLVRY_RISK_DECILE', 5)
    risk_percentile = risk_info.get('DLVRY_RISK_PERCENTILE', 50)
    top_features = risk_info.get('DLVRY_RISK_TOP_FEATURE', '').split(',')
    
    # Return in the format expected by the pipeline
    return {
        "status": "success",
        "risk_assessment": {
            "overall_risk_score": risk_decile,
            "risk_level": risk_bucket,
            "risk_percentile": risk_percentile,
            "top_risk_factors": top_features,
            "model_version": "bigquery_precalculated_v1"
        }
    }

print("✅ External model integration configured")
print("💡 Using pre-calculated risk scores from BigQuery data")

## Additional Risk Assessment Tools

While your external model provides the primary risk assessment, we can enhance it with specific risk factors. These additional assessments can:
- Provide more granular insights
- Validate the external model's assessment
- Identify risks the general model might miss

Let's create specialized risk assessment tools:

In [ ]:
def assess_weather_risk(environmental_data: Dict[str, Any]) -> Dict[str, Any]:
    """Assess weather-related delivery risks"""
    precipitation = environmental_data.get('PRECIPITATION', 0)
    weather_category = environmental_data.get('WTHR_CATEGORY', 'Unknown')
    
    # Simple weather risk scoring
    risk_score = 0
    factors = []
    
    # Handle precipitation as string (e.g., "0.09 inch")
    if isinstance(precipitation, str):
        precipitation = float(precipitation.replace(' inch', ''))
    
    if precipitation > 0.5:
        risk_score = 8
        factors.append(f"Heavy precipitation ({precipitation} inches)")
    elif precipitation > 0.1:
        risk_score = 4
        factors.append(f"Light precipitation ({precipitation} inches)")
    else:
        risk_score = 2
        factors.append("Favorable weather")
    
    if weather_category.lower() in ['rain', 'snow', 'storm']:
        risk_score = min(10, risk_score + 3)
        factors.append(f"Adverse weather: {weather_category}")
    
    return {
        "weather_risk_score": risk_score,
        "weather_factors": factors,
        "precipitation_inches": precipitation,
        "category": weather_category
    }


def assess_customer_risk(customer_data: Dict[str, Any]) -> Dict[str, Any]:
    """Assess customer-related delivery risks"""
    risk_score = 5  # Base score
    factors = []
    
    # PRO customers typically have lower risk
    if customer_data.get('PRO_XTRA_MEMBER', False):
        risk_score -= 2
        factors.append("PRO customer (lower risk)")
    
    # Commercial vs residential
    if customer_data.get('COMMERCIAL_ADDRESS_FLAG', False):
        risk_score -= 1
        factors.append("Commercial address")
    else:
        risk_score += 2
        factors.append("Residential address")
    
    # Customer notes indicate special requirements
    if customer_data.get('CUSTOMER_NOTES'):
        risk_score += 2
        factors.append("Special delivery instructions")
        
    return {
        "customer_risk_score": max(1, min(10, risk_score)),
        "customer_factors": factors,
        "has_special_instructions": bool(customer_data.get('CUSTOMER_NOTES'))
    }


def assess_route_risk(order_data: Dict[str, Any], environmental_data: Dict[str, Any]) -> Dict[str, Any]:
    """Assess route and accessibility risks"""
    risk_score = 5  # Base score
    factors = []
    
    # Vehicle type considerations
    vehicle_type = order_data.get('VEHICLE_TYPE', 'UNKNOWN')
    if vehicle_type == 'FLAT' and order_data.get('WEIGHT', 0) > 1000:
        risk_score += 2
        factors.append("Heavy load on flatbed")
    
    # Street view analysis
    street_desc = environmental_data.get('STRT_VW_IMG_DSCRPTN', '')
    if 'dead end' in street_desc.lower():
        risk_score += 2
        factors.append("Dead end street")
    if 'limited' in street_desc.lower() or 'narrow' in street_desc.lower():
        risk_score += 1
        factors.append("Access limitations noted")
    
    return {
        "route_risk_score": max(1, min(10, risk_score)),
        "route_factors": factors,
        "access_notes": street_desc
    }

print("✅ Risk assessment tools created")

## Building the Risk Assessment Pipeline

Now let's create agents that use these tools and orchestrate them into a complete risk assessment pipeline:

1. **External Risk Agent**: Calls your proprietary model (simulated with pre-calculated BigQuery data)
2. **Weather Risk Agent**: Uses MCP to get weather risk assessment from our weather service!
3. **Customer Risk Agent**: Assesses customer-specific risks
4. **Route Risk Agent**: Assesses route and accessibility risks

### MCP Integration for Weather

The weather agent now uses `MCPToolset` to connect to our weather MCP server. This demonstrates how easily you can integrate external services into your ADK agents:

In [ ]:
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StdioServerParameters
import sys

GEMINI_MODEL = "gemini-2.0-flash"

# Agent that calls your external risk model
external_risk_agent = Agent(
    model=GEMINI_MODEL,
    name="external_risk_agent",
    description="Integrates with external risk assessment model",
    instruction="""\
You will receive consolidated order data in the user message.
Parse the JSON order data from the user message and use the call_external_risk_model tool to get risk assessment from the external model.
Pass the entire order data structure to the model and return its assessment.
This demonstrates integration with external AI models - in production, this would call the client's proprietary risk model.
""",
    tools=[call_external_risk_model],
    output_key="external_risk_assessment"
)

# Weather risk agent - NOW WITH MCP INTEGRATION!
weather_risk_agent = Agent(
    model=GEMINI_MODEL,
    name="weather_risk_agent",
    description="Assesses weather-related delivery risks using MCP weather service",
    instruction="""\
You will receive order data in the user message.

Parse the JSON order data and:
1. Extract the delivery date from order.SCHEDULED_DELIVERY_DATE (format: "2025-06-21T00:00:00")
2. Extract the city - for Chicago area deliveries, use "Chicago"
3. Use the 'assess_weather_risk' MCP tool with the city and date

The tool will return structured risk data including:
- weather_risk_score (1-10)
- weather_factors (list of risk factors)
- weather_data (temperature, conditions, precipitation, etc.)
- risk_level (HIGH/MEDIUM/LOW)

Return the complete response from the MCP tool.
""",
    tools=[
        MCPToolset(
            connection_params=StdioServerParameters(
                # Use Python to run the weather MCP server
                command=sys.executable,
                args=[os.path.abspath("./weather_mcp_server.py")],
            ),
            # Filter to only expose the risk assessment tool
            tool_filter=['assess_weather_risk']
        )
    ],
    output_key="weather_risk"
)

# Customer risk agent  
customer_risk_agent = Agent(
    model=GEMINI_MODEL,
    name="customer_risk_agent",
    description="Assesses customer-related delivery risks",
    instruction="""\
You will receive order data in the user message.
Parse the JSON order data, extract the customer data and use the assess_customer_risk tool to evaluate customer-related delivery risks.
Consider PRO status, special instructions, and address type.
""",
    tools=[assess_customer_risk],
    output_key="customer_risk"
)

# Route risk agent
route_risk_agent = Agent(
    model=GEMINI_MODEL,
    name="route_risk_agent",
    description="Assesses route and accessibility risks",
    instruction="""\
You will receive order data in the user message.
Parse the JSON order data, extract order and environmental data, then use the assess_route_risk tool to evaluate route-related delivery risks.
Consider load weight, vehicle requirements, and street accessibility.
""",
    tools=[assess_route_risk],
    output_key="route_risk"
)

print("✅ Risk assessment agents created")
print("\n🔍 Key MCP Integration Points:")
print("1. MCPToolset wraps the MCP connection")
print("2. StdioServerParameters defines how to connect (command + args)")
print("3. tool_filter lets you select specific tools from the MCP server")
print("4. The agent uses MCP tools just like regular ADK tools!")

## Orchestrating Risk Assessments

We'll run the additional risk assessments in parallel for efficiency:

In [ ]:
# Parallel execution of additional risk assessments
additional_risks_agent = ParallelAgent(
    name="additional_risks_agent",
    sub_agents=[weather_risk_agent, customer_risk_agent, route_risk_agent],
    description="Assess multiple risk factors in parallel"
)

print("✅ Parallel risk assessments configured")

In [ ]:
# Final risk aggregation agent - outputs structured JSON
risk_aggregation_agent = Agent(
    name="risk_aggregation_agent",
    model=GEMINI_MODEL,
    description="Aggregates all risk assessments into structured output",
    instruction="""\
You have received multiple risk assessments:

External Model Assessment: {external_risk_assessment}
Weather Risk: {weather_risk}
Customer Risk: {customer_risk}
Route Risk: {route_risk}

Create a structured JSON output that includes:

{
    "risk_assessment": {
        "overall_risk_score": <external model decile>,
        "risk_level": "<HIGH/MEDIUM/LOW from external model>",
        "risk_percentile": <external model percentile>,
        "risk_scores": {
            "overall": <external model decile>,
            "weather": <weather_risk_score>,
            "customer": <customer_risk_score>,
            "route": <route_risk_score>
        },
        "risk_factors": [
            // Array of all identified risk factors from all assessments
        ],
        "top_risks": "<external model top features>",
        "recommendations": [
            {
                "action": "<specific action>",
                "priority": "<HIGH/MEDIUM/LOW>",
                "reason": "<why this is needed>"
            }
        ],
        "weather_data": {
            // Include weather assessment details
        }
    }
}

Return ONLY the JSON structure, no additional text or markdown.
""",
    tools=[],
    output_key="risk_assessment_data"
)

print("✅ Risk aggregation agent created")

## Building the Complete Pipeline

Now let's assemble all the agents into a sequential pipeline:

In [ ]:
# Full risk assessment pipeline
risk_assessment_pipeline = SequentialAgent(
    name="risk_assessment_pipeline",
    sub_agents=[external_risk_agent, additional_risks_agent, risk_aggregation_agent],
    description="Complete risk assessment pipeline with external model integration and real weather data"
)

print("✅ Risk assessment pipeline assembled")
print("\nPipeline flow:")
print("1️⃣  External Risk Model")
print("2️⃣  Parallel: Weather + Customer + Route Analysis")  
print("3️⃣  Risk Aggregation → Structured JSON Output")

## Running the Complete Risk Assessment

Now let's run the pipeline with our actual order data from Exercise 1:

In [ ]:
# Run the risk assessment pipeline
import asyncio
import sys
import io

async def run_risk_assessment_demo():
    """Run risk assessment pipeline on the loaded order data"""
    
    # Setup
    session_service = InMemorySessionService()
    await session_service.create_session(
        app_name="risk_assessment",
        user_id="user_1",
        session_id="risk_session_001"
    )
    
    runner = Runner(
        agent=risk_assessment_pipeline,
        app_name="risk_assessment",
        session_service=session_service
    )
    
    print("=" * 60)
    print("DELIVERY RISK ASSESSMENT PIPELINE")
    print("=" * 60)
    print(f"\nAssessing risk for order: {order_data.get('order', {}).get('CUSTOMER_ORDER_NUMBER', 'Unknown')}")
    print(f"Customer: {order_data.get('customer', {}).get('CUSTOMER_NAME', 'Unknown')}")
    print(f"Risk Level (Pre-calculated): {order_data.get('risk_info', {}).get('DLVRY_RISK_BUCKET', 'Unknown')}")
    print("\nRunning comprehensive risk assessment...\n")
    
    # Create message
    content = types.Content(
        role="user",
        parts=[types.Part(text=json.dumps(order_data))]
    )
    
    # Run pipeline with output suppression
    old_stderr = sys.stderr
    sys.stderr = io.StringIO()
    
    try:
        async for event in runner.run_async(
            user_id="user_1",
            session_id="risk_session_001",
            new_message=content
        ):
            sys.stderr = old_stderr
            
            if hasattr(event, "author") and event.author:
                if event.author in ["external_risk_agent", "weather_risk_agent", 
                                  "customer_risk_agent", "route_risk_agent"]:
                    print(f"[{event.author}] analyzing...")
                    
            if event.is_final_response() and event.author == "risk_aggregation_agent":
                if event.content and event.content.parts:
                    print("\n" + "=" * 60)
                    print("RISK ASSESSMENT COMPLETE")
                    print("=" * 60)
                    
                    # Parse and display JSON
                    try:
                        response_text = event.content.parts[0].text.strip()
                        if response_text.startswith("```json"):
                            response_text = response_text[7:]
                        if response_text.endswith("```"):
                            response_text = response_text[:-3]
                        
                        risk_data = json.loads(response_text.strip())
                        
                        # Save to file
                        with open('risk_assessment_output.json', 'w') as f:
                            json.dump(risk_data, f, indent=2)
                        
                        # Display key results
                        assessment = risk_data.get('risk_assessment', {})
                        print(f"\n📊 Overall Risk Score: {assessment.get('overall_risk_score')}/10")
                        print(f"⚠️  Risk Level: {assessment.get('risk_level')}")
                        print(f"📈 Risk Percentile: {assessment.get('risk_percentile')}%")
                        
                        print("\n🔍 Risk Scores by Category:")
                        scores = assessment.get('risk_scores', {})
                        print(f"   - Weather: {scores.get('weather')}/10")
                        print(f"   - Customer: {scores.get('customer')}/10")
                        print(f"   - Route: {scores.get('route')}/10")
                        
                        print("\n⚡ Top Risk Factors:")
                        for factor in assessment.get('risk_factors', [])[:5]:
                            print(f"   - {factor}")
                        
                        print("\n💡 Recommendations:")
                        for rec in assessment.get('recommendations', [])[:3]:
                            print(f"   - [{rec['priority']}] {rec['action']}")
                            print(f"     Reason: {rec['reason']}")
                        
                        print("\n✅ Risk assessment saved to risk_assessment_output.json")
                        
                        return risk_data
                        
                    except Exception as e:
                        print(f"Error parsing JSON: {e}")
                        print(event.content.parts[0].text)
                break
                
            sys.stderr = io.StringIO()
    finally:
        sys.stderr = old_stderr

# Run the demonstration
result = await run_risk_assessment_demo()

## Understanding the MCP Output

Let's examine what the weather MCP server returned:

```json
{
  "city": "Chicago",
  "date": "2025-06-21",
  "weather_risk_score": 1,
  "weather_factors": ["Favorable weather conditions"],
  "weather_data": {
    "temperature": 72,
    "conditions": "Partly Cloudy",
    "precipitation": 0.0,
    "wind_speed": 8,
    "humidity": 65
  },
  "risk_level": "LOW",
  "mode": "demo"
}
```

The MCP server:
1. Received the city and date from our weather agent
2. Assessed the weather conditions (currently using demo data)
3. Calculated a risk score based on precipitation, wind, and conditions
4. Returned structured data that our pipeline can use

In production, you would:
- Set `OPENWEATHER_API_KEY` environment variable for real weather data
- Deploy the MCP server as a microservice
- Use caching to reduce API calls
- Add authentication for security

## Structured Output for Pipeline Integration

The risk assessment now outputs structured JSON data that includes:

- **Risk Scores**: Overall and category-specific scores (1-10)
- **Risk Level**: HIGH/MEDIUM/LOW classification
- **Risk Factors**: Detailed list of identified risks
- **Recommendations**: Actionable items with priorities
- **Weather Data**: Real-time or simulated weather information

This structured output (`risk_assessment_output.json`) can be:
1. Combined with order data from the previous pipeline
2. Used to calculate priority scores
3. Fed into communication generation systems
4. Consumed by UI applications

The modular design ensures each pipeline stage produces reusable, structured data!

## Key Takeaways

This exercise demonstrated several important patterns:

### 1. **External Model Integration**
- Easy to integrate existing AI models into ADK workflows
- Used pre-calculated risk scores from BigQuery data
- Simple to swap between different model providers

### 2. **MCP (Model Context Protocol) Integration**
- Connected to external weather service via MCP
- Weather agent uses `MCPToolset` to call the weather risk assessment
- MCP server provides standardized interface for external services
- Easy to switch between demo mode and real weather APIs

### 3. **Multi-Factor Risk Analysis**
- Parallel agents assess different risk dimensions
- Specialized tools for specific risk factors
- Comprehensive view beyond single model assessment

### 4. **Production Considerations**
```python
# Instead of our demo model:
model = GenerativeModel("gemini-1.5-flash")

# You would use:
# Option 1: Your proprietary model endpoint
response = requests.post("https://your-model.api/assess", json=order_data)

# Option 2: Claude from Model Garden
model = GenerativeModel("claude-3-sonnet@20240229")

# Option 3: Your custom Vertex AI model
endpoint = aiplatform.Endpoint("your-endpoint-id")
response = endpoint.predict(instances=[order_data])
```

### 5. **Actionable Insights**
- Not just risk scores, but specific recommendations
- Comparison between models for validation
- Clear prioritization of mitigation actions

### 6. **Benefits of MCP**
- **Standardized Interface**: All MCP servers follow the same protocol
- **Language Agnostic**: Can use MCP servers written in any language
- **Easy Integration**: MCPToolset makes it simple to add external services
- **Flexibility**: Switch between local development and production services

## Next Steps

In the next exercise, we'll take these risk assessments and:
1. Analyze product characteristics for special handling needs
2. Calculate priority scores based on multiple factors
3. Generate intelligent insights for delivery optimization

The modular design means you can easily:
- Replace the external model with your own
- Add new risk factors
- Customize the output format
- Integrate with your existing systems
- Switch MCP servers for different weather providers

## Try It Yourself!

> **⚠️ Important**: Complete all 5 exercises before trying these modifications! The exercises build on each other's outputs. Changing code here will affect the data flow to subsequent exercises.

Now it's your turn to experiment with the risk assessment pipeline:

### Exercise 1: Adjust Risk Thresholds
Modify the `assess_weather_risk` function to change the precipitation thresholds:
```python
# Current: risk_score = 8 if precipitation > 0.5
# Try: risk_score = 6 if precipitation > 0.3
```
Run the pipeline again and see how it affects the overall risk assessment.

### Exercise 2: Add a New Risk Factor
Add code to check if the delivery window is during rush hour (7-9 AM or 4-6 PM):
```python
# In assess_route_risk function, check:
window_start = order_data.get('WINDOW_START')
# If window overlaps with rush hour, add 2 to risk_score
```

### Exercise 3: Customize Recommendations
In the risk_aggregation_agent instruction, add logic to generate specific recommendations based on risk scores:
- If weather_risk > 6: "Schedule delivery for better weather day"
- If customer has special notes: "Call customer before delivery"
- If route_risk > 7: "Use experienced driver familiar with area"

### Challenge: Create Your Own MCP Server
Create a simple MCP server that provides traffic data:
1. Copy the weather_mcp_server.py as a template
2. Create a "get_traffic_conditions" tool
3. Return traffic congestion level (1-10) based on time of day
4. Integrate it into the risk assessment pipeline

### Important: Session State Setup

Notice that we set the session state **before** creating the Runner. This is crucial because:

1. The ADK framework attempts to substitute template variables (like `{order_data}`) when agents are initialized
2. If the session state isn't populated yet, you'll get a `KeyError` about missing context variables
3. By setting the state first, the agents can properly access the data during initialization

In the current implementation, we've also updated the agent instructions to parse data from the user message instead of relying solely on template substitution, making the system more robust.